# Code Buddy: Your Technical Code Assistant
Made by: Guru Deep Singh

Code Buddy is an LLM powered assistant allowing you to ask it questions regarding your coding language issues

In [1]:
#imports
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display

In [2]:
# Class for creating a LLM powered assistant

class CodeBuddy:
    ALLOWED_BACKENDS = {"ollama", "openai"}
    def __init__(self, backend="ollama", modelName= "llama3.2"):
        if backend not in self.ALLOWED_BACKENDS:
            raise ValueError(f"Invalid backend '{backend}'. Allowed options are: {', '.join(self.ALLOWED_BACKENDS)}")
           
        self.backend = backend
        self.modelName = modelName
        print(f"Code Buddy using {self.backend} backend and {modelName} model")
        self.__initialize()

    def __initialize(self):
        if self.backend == "openai":
            load_dotenv(override=True)
            apiKey = os.getenv('OPENAI_API_KEY')
            if not apiKey:
                print("No API key was found")

            else:
                self.__requestHandler = OpenAI()

        elif self.backend == "ollama":
            # Setting up Constants
            self._OLLAMA_API = "http://localhost:11434/api/chat"
            self._HEADERS = {"Content-Type": "application/json"}
            self.__requestHandler = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
            
    def __nonStreamCall(self,userPrompt, systemPrompt):
        response = self.__requestHandler.chat.completions.create(
            model = self.modelName,
            messages = [
                {"role": "system", "content": systemPrompt},
                {"role": "user", "content": userPrompt}
                ])
        return response.choices[0].message.content

    def __streamCall(self,userPrompt, systemPrompt):
        stream = self.__requestHandler.chat.completions.create(
                model = self.modelName,
                messages = [
                    {"role": "system", "content": systemPrompt},
                    {"role": "user", "content": userPrompt}
                ],
                stream=True
            )
            
        for chunk in stream:
            response = chunk.choices[0].delta
            if hasattr(response, "content"):
                yield response.content



    def run(self, userPrompt, stream = False, 
            systemPrompt ="You are a helpful and informed coding agent.\
            You are given a piece of code. You have to check if the code is correct or is incorrect.\
            You need to explain the code in beginner friendy way.\
            You are also allowed to give suggestions on improvement of code for runtime optimization.\
            Give your answer in Markdown." ):
        
        if len(systemPrompt.strip()) != 0 and len(userPrompt.strip()) != 0:
            if not stream:
                return self.__nonStreamCall(userPrompt, systemPrompt)

            else:
                return self.__streamCall(userPrompt, systemPrompt)
                
            
    

        

In [ ]:
# Helper functions for streaming
def stream_markdown(generator):
    buffer = ""
    display_handle = display(Markdown(""), display_id=True)

    for part in generator:
        buffer += str(part)
        cleaned = buffer.replace("```", "").replace("markdown", "")
        update_display(Markdown(cleaned), display_id=display_handle.display_id)

def stream(generator):
    for part in generator:
        print(part, end="", flush=True)

In [ ]:
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""
# Assistant powered by OpenAI API 
myBuddy = CodeBuddy("openai", "gpt-4o-mini")


In [ ]:
output = myBuddy.run(userPrompt = question, stream=False)
display(Markdown(output))

In [ ]:
stream_markdown(myBuddy.run(userPrompt=question, stream=True))

# Ollama based Code Buddy

In [ ]:
# Assistant powered by Ollama 
myBuddyOllama = CodeBuddy("ollama", "llama3.2")

In [ ]:
# Pulling model
!ollama pull llama3.2

In [ ]:
outputOllama = myBuddyOllama.run(userPrompt = question, stream=False)
display(Markdown(outputOllama))

In [ ]:
stream_markdown(myBuddyOllama.run(userPrompt=question, stream=True))

# Creating a GUI with Gradio

In [3]:
import gradio as gr

In [4]:
# Gradio function call 
def runModel(model, userPrompt):
    if model == "llama3.2":
        !ollama pull llama3.2
        stream = CodeBuddy("ollama", model).run(userPrompt, True)
    elif model == "gpt-4o-mini":
        gr.Warning("Paid Service being Used")
        stream = CodeBuddy("openai", model).run(userPrompt, True)
    else:
        raise ValueError("Model not supported")
    result = ""
    for part in stream:
        result += str(part)
        yield result

In [6]:
interface = gr.Interface(fn=runModel, inputs=[gr.Dropdown(["llama3.2", "gpt-4o-mini"], label="Model", value="llama3.2"), gr.Textbox(label="Enter your Coding question here")],
                        outputs=gr.Markdown(label="Response:"),
                        flagging_mode="never")
interface.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
